## Get data from NSE

In [1]:



import requests
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
fromdate = datetime.strftime(datetime.today(),'%d-%m-%Y')
todate =  datetime.today() - timedelta(days=120)  
enddate = datetime.strftime(todate,'%d-%m-%Y')
url = 'https://www.nseindia.com/api/corporates-pit?index=equities&from_date='+ enddate+ '&to_date=' + fromdate
print(url)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://www.nseindia.com',
    'X-Requested-With': 'XMLHttpRequest'
}


session = requests.Session()
session.headers.update(headers)


session.get('https://www.nseindia.com',timeout=5)  # Initial request to get the cookies


response = session.get(url)
data = response.json()
data

ModuleNotFoundError: No module named 'requests'

## Convert Json to Dataframe

In [ ]:
df = pd.DataFrame(data['data'])
df

In [ ]:
df.replace("-", 0, inplace=True)

In [ ]:

df["secVal"] = pd.to_numeric(df["secVal"])
df['acqfromDt'] = pd.to_datetime(df['acqfromDt'],format='mixed')
df

In [ ]:

personCat = ['Promoters','Promoter Group']
df = df[(df['personCategory'].isin(personCat)) & (df['acqMode'] == 'Market Purchase') & (df['tdpTransactionType'] == 'Buy') ]
df

In [ ]:

df[df['symbol'] == "POONAWALLA"]

In [ ]:

df1 = df.groupby(['symbol']).agg({'secVal':'sum','acqfromDt':'max'}).reset_index()
df1

In [ ]:
df1 = df1.sort_values(by = "acqfromDt",ascending = False)
df1

In [ ]:
df1.head(50)